In [2]:
import json
import random
from collections import defaultdict

In [3]:
train_data_file = "/opt/ml/repo/taepd/input/data/train_dataset/train_dials.json"

In [4]:
data = json.load(open(train_data_file))

In [4]:
data[0]

{'dialogue_idx': 'snowy-hat-8324:관광_식당_11',
 'domains': ['관광', '식당'],
 'dialogue': [{'role': 'user',
   'text': '서울 중앙에 있는 박물관을 찾아주세요',
   'state': ['관광-종류-박물관', '관광-지역-서울 중앙']},
  {'role': 'sys',
   'text': '안녕하세요. 문화역서울 284은 어떠신가요? 평점도 4점으로 방문객들에게 좋은 평가를 받고 있습니다.'},
  {'role': 'user',
   'text': '좋네요 거기 평점은 말해주셨구 전화번호가 어떻게되나요?',
   'state': ['관광-종류-박물관', '관광-지역-서울 중앙', '관광-이름-문화역서울 284']},
  {'role': 'sys', 'text': '전화번호는 983880764입니다. 더 필요하신 게 있으실까요?'},
  {'role': 'user',
   'text': '네 관광지와 같은 지역의 한식당을 가고싶은데요 야외석이 있어야되요',
   'state': ['관광-종류-박물관',
    '관광-지역-서울 중앙',
    '관광-이름-문화역서울 284',
    '식당-지역-서울 중앙',
    '식당-종류-한식당',
    '식당-야외석 유무-yes']},
  {'role': 'sys', 'text': '생각하고 계신 가격대가 있으신가요?'},
  {'role': 'user',
   'text': '음.. 저렴한 가격대에 있나요?',
   'state': ['관광-종류-박물관',
    '관광-지역-서울 중앙',
    '관광-이름-문화역서울 284',
    '식당-가격대-저렴',
    '식당-지역-서울 중앙',
    '식당-종류-한식당',
    '식당-야외석 유무-yes']},
  {'role': 'sys', 'text': '죄송하지만 저렴한 가격대에는 없으시네요.'},
  {'role': 'user',
   'text': '그럼 비싼 가격대로 다시

In [5]:
num_data = len(data)
num_data

7000

In [6]:
num_dev = int(num_data * 0.1)
num_dev

700

In [7]:
dom_mapper = defaultdict(list)
for d in data:
    dom_mapper[len(d["domains"])].append(d["dialogue_idx"])

In [8]:
print(dom_mapper.keys())
print(len(dom_mapper))
print(list(dom_mapper.values())[0][:5])

dict_keys([2, 1, 3])
3
['snowy-hat-8324:관광_식당_11', 'calm-smoke-9954:관광_식당_7', 'wild-bonus-5601:식당_택시_12', 'bitter-king-4453:숙소_택시_12', 'old-mountain-0099:숙소_택시_7']


In [35]:
num_per_domain_transition = int(num_dev / 3)
num_per_domain_transition

233

In [51]:
dev_idx = []
for v in dom_mapper.values():
        idx = random.sample(v, num_per_domain_transition)
        print((idx)[:10])
        dev_idx.extend(idx)

['steep-frog-4363:관광_택시_10', 'bold-sea-9536:식당_택시_10', 'round-dawn-6071:관광_숙소_8', 'autumn-violet-9413:관광_지하철_3', 'odd-flower-6966:숙소_택시_8', 'rough-star-9505:식당_택시_9', 'wandering-fire-3190:관광_택시_9', 'super-king-5654:숙소_식당_12', 'rapid-mud-1478:식당_숙소_11', 'throbbing-flower-6815:관광_지하철_3']
['tight-wind-4387:택시_1', 'green-bread-9930:택시_1', 'soft-frost-4544:식당_11', 'quiet-block-9676:관광_8', 'winter-moon-2287:숙소_2', 'summer-field-9959:관광_26', 'billowing-leaf-1239:식당_34', 'long-star-4427:관광_27', 'muddy-bonus-4927:식당_15', 'wild-snowflake-3434:숙소_6']
['lucky-water-6973:식당_숙소_택시_11', 'silent-breeze-0564:숙소_식당_택시_13', 'withered-glade-0971:관광_숙소_식당_10', 'ancient-violet-9767:식당_숙소_택시_13', 'still-unit-7784:숙소_식당_지하철_15', 'empty-dew-3781:숙소_식당_관광_15', 'orange-band-1279:관광_숙소_지하철_14', 'broad-snowflake-8143:숙소_식당_관광_15', 'calm-bonus-4969:식당_관광_택시_13', 'long-leaf-7092:식당_숙소_관광_15']


In [53]:
train_data, dev_data = [], []
for d in data:
    if d["dialogue_idx"] in dev_idx:
        dev_data.append(d)
    else:
        train_data.append(d)

In [55]:
len(train_data)

6301

In [59]:
dev_labels = {}
for dialogue in dev_data:
    d_idx = 0
    guid = dialogue["dialogue_idx"]
    for idx, turn in enumerate(dialogue["dialogue"]):
        if turn["role"] != "user":
            continue

        state = turn.pop("state")

        guid_t = f"{guid}-{d_idx}"
        d_idx += 1

        dev_labels[guid_t] = state

In [60]:
dev_labels

{'broad-bar-0649:식당_택시_11-0': ['식당-가격대-dontcare', '식당-지역-서울 중앙', '식당-종류-한식당'],
 'broad-bar-0649:식당_택시_11-1': ['식당-가격대-dontcare',
  '식당-지역-서울 중앙',
  '식당-종류-한식당',
  '식당-예약 요일-월요일',
  '식당-예약 시간-19:40',
  '식당-예약 명수-1',
  '식당-이름-꼬끼오꼬꼬'],
 'broad-bar-0649:식당_택시_11-2': ['식당-가격대-dontcare',
  '식당-지역-서울 중앙',
  '식당-종류-한식당',
  '식당-예약 요일-월요일',
  '식당-예약 시간-19:40',
  '식당-예약 명수-1',
  '식당-이름-꼬끼오꼬꼬'],
 'broad-bar-0649:식당_택시_11-3': ['식당-가격대-dontcare',
  '식당-지역-서울 중앙',
  '식당-종류-한식당',
  '식당-예약 요일-월요일',
  '식당-예약 시간-19:40',
  '식당-예약 명수-1',
  '식당-이름-꼬끼오꼬꼬',
  '택시-출발 시간-03:00',
  '택시-출발지-종각역',
  '택시-도착지-꼬끼오꼬꼬'],
 'broad-bar-0649:식당_택시_11-4': ['식당-가격대-dontcare',
  '식당-지역-서울 중앙',
  '식당-종류-한식당',
  '식당-예약 요일-월요일',
  '식당-예약 시간-19:40',
  '식당-예약 명수-1',
  '식당-이름-꼬끼오꼬꼬',
  '택시-출발 시간-03:00',
  '택시-출발지-종각역',
  '택시-도착지-꼬끼오꼬꼬',
  '택시-종류-dontcare'],
 'broad-bar-0649:식당_택시_11-5': ['식당-가격대-dontcare',
  '식당-지역-서울 중앙',
  '식당-종류-한식당',
  '식당-예약 요일-월요일',
  '식당-예약 시간-19:40',
  '식당-예약 명수-1',
  '식당-이름-꼬끼오꼬꼬',
  '택시-출발 시간-03:00',
  '택